In [1]:
import warnings
warnings.filterwarnings("ignore") #never print matching warnings
from collections import Counter
import pandas as pd
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import math
import time
import re
import os
nltk.download('stopwords') 
from tqdm import tqdm
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import numpy as np
import seaborn as sns
sns.set_style('whitegrid')

[nltk_data] Downloading package stopwords to /home/amwin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.naive_bayes as naive

In [70]:
df = pd.read_csv("Training_Data.csv",encoding='latin-1',index_col=0)

In [56]:
df.head(5)

,Sentence,Growth,Inflation,Cad,Money,Fiscal,Other
0,Profits of listed Indian companies have remain...,0,0,0,0,0,0
1,"On the other hand, nominal gross domestic prod...",0,0,0,0,0,0
2,The upshot is corporate earnings expressed as ...,0,0,0,0,0,0
3,Corporate profit as a percentage of GDP for th...,0,0,0,0,0,0
4,"In fact, the measure has consistently declined...",0,0,0,0,0,0


In [5]:
stop_words = set(stopwords.words('english'))

def nlp_preprocessing(total_text, index, column):
    if type(total_text) is not int:
        total_text = re.sub(r'[^\x00-\x7f]',r'', total_text) 
        total_text = re.sub('[^A-Za-z]+', ' ', total_text)
        total_text = ' '.join(e.lower() for e in total_text.split() if e.lower() not in stop_words)
        df[column][index] = total_text
    
start_time = time.clock()
for index, row in tqdm(df.iterrows()):
    nlp_preprocessing(row['Sentence'],index,'Sentence')
print(time.clock() - start_time, "seconds")

220853it [28:12, 130.49it/s]

1694.7939119999999 seconds


In [6]:
columns = ['Sentence', 'Label']

In [9]:
Data = pd.DataFrame(columns=columns)
New_Data = pd.DataFrame(columns=['Sentence'])

In [10]:
count = 0
for index, row in tqdm(df.iterrows()):
    if row['Growth'] == 1:
        Data = Data.append({'Sentence':row['Sentence'],'Label':1,'Index': count},ignore_index=True)
    elif row['Inflation'] == 1:
        Data = Data.append({'Sentence':row['Sentence'],'Label':2,'Index': count},ignore_index=True)
    elif row['Cad'] == 1:
        Data = Data.append({'Sentence':row['Sentence'],'Label':3,'Index': count},ignore_index=True)
    elif row['Money'] == 1:
        Data = Data.append({'Sentence':row['Sentence'],'Label':4,'Index': count},ignore_index=True)
    elif row['Fiscal'] == 1:
        Data = Data.append({'Sentence':row['Sentence'],'Label':5,'Index': count},ignore_index=True)
    elif row['Other'] == 1:
        Data = Data.append({'Sentence':row['Sentence'],'Label':6,'Index': count},ignore_index=True)
    else:
        New_Data = New_Data.append({'Sentence':row['Sentence'],'Index': count},ignore_index=True)
    count = count + 1

220853it [18:08, 202.95it/s]


In [25]:
labels = Data['Label'].to_numpy().astype('int')
labels

array([2, 1, 2, ..., 3, 3, 3])

In [12]:
New_Data.to_csv('UnLabelled_Data.csv', index = False)
Data.to_csv('Labelled_Data.csv', index = False)

In [7]:
#train_data_text, test_data_text,train_data_class,test_data_class = train_test_split(Data['Sentence'], Data['Label'], test_size = 0.2, random_state=42)

In [15]:
# text_vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, smooth_idf=True) #(sublinear_tf=True, max_df=0.5, stop_words='english'), https://towardsdatascience.com/training-a-naive-bayes-model-to-identify-the-author-of-an-email-or-document-17dc85fa630a
# features_train   = text_vectorizer.fit_transform(Data['Sentence'])
# features_test = text_vectorizer.transform(New_Data['Sentence'])

In [14]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
features_train = tf_idf_vect.fit_transform(Data['Sentence'])
features_test = tf_idf_vect.transform(New_Data['Sentence'])

In [ ]:
# tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
# features_train = tf_idf_vect.fit_transform(train_data_text)
# features_test = tf_idf_vect.transform(test_data_text)

In [28]:
# alpha_list = list()
# for i in range(1,20,1):
#   alpha_list.append(i)

# from sklearn.model_selection import GridSearchCV
# tuned_parameter = [{'alpha':alpha_list}]
# clf = GridSearchCV(naive.MultinomialNB(), tuned_parameter,cv = 5, scoring='accuracy' )
# clf.fit(features_train, train_data_class)
# print(clf.best_params_)

{'alpha': 1}


In [29]:
# preds = clf.predict(features_test)
# print(preds)
# acc = accuracy_score(test_data_class, preds) * 100
# print('\nThe accuracy of multinomial Naive Bayes classifier is %f%%' % (acc))

[1 2 1 ... 1 1 2]

The accuracy of multinomial Naive Bayes classifier is 88.807398%


In [ ]:
# preds = clf.predict(features_test)

In [26]:
svc = svm.SVC()
svc.fit(features_train,labels)
prediction_svc = svc.predict(features_test)

In [27]:
New_Data['Label'] = prediction_svc

In [59]:
frames = [Data, New_Data]
result = pd.concat(frames)


In [64]:
result['Index'] = result.astype({"Index": int})

In [65]:
result.sort_values(by=['Index'], inplace = True)
result.set_index('Index', inplace = True)

In [71]:
for index, row in result.iterrows():
    if row['Label'] == 1:
        df.set_value(index,'Growth',1)
    elif row['Label'] == 2:
        df.set_value(index,'Inflation',1)
    elif row['Label'] == 3:
        df.set_value(index,'Cad',1)
    elif row['Label'] == 4:
        df.set_value(index,'Money',1)
    elif row['Label'] == 5:
        df.set_value(index,'Fiscal',1)
    elif row['Label'] == 6:
        df.set_value(index,'Other',1)


In [73]:
df.head(10)

,Sentence,Growth,Inflation,Cad,Money,Fiscal,Other
0,Profits of listed Indian companies have remain...,1,0,0,0,0,0
1,"On the other hand, nominal gross domestic prod...",1,0,0,0,0,0
2,The upshot is corporate earnings expressed as ...,1,0,0,0,0,0
3,Corporate profit as a percentage of GDP for th...,1,0,0,0,0,0
4,"In fact, the measure has consistently declined...",1,0,0,0,0,0
5,"Whats more, it is also lower than the long-pe...",1,0,0,0,0,0
6,Aggregate profit of Indian firms is likely to ...,1,0,0,0,0,0
7,"In addition to weak demand, the sharp fall in ...",0,1,0,0,0,0
8,"What offers comfort, however, is that things a...",1,0,0,0,0,0
9,There could be improvement in the days to com...,1,0,0,0,0,0


In [74]:
df.to_csv('Output.csv', index = False)